In [12]:
from estnltk import Text
from estnltk.storage.postgres import PostgresStorage, create_schema

storage = PostgresStorage(host='postgres.keeleressursid.ee',
                          port=5432,
                          dbname='estonian-text-corpora',
                          user='rasmusm',
                          password='',
                          schema='estonian_text_corpora',
                          role='estonian_text_corpora_read',
                          temporary=False)

INFO:storage.py:42: connecting to host: 'postgres.keeleressursid.ee', port: 5432, dbname: 'estonian-text-corpora', user: 'rasmusm'
INFO:storage.py:58: schema: 'estonian_text_corpora', temporary: False, role: 'estonian_text_corpora_read'


In [13]:
collection = storage['koondkorpus_base_subset_of_5000_v2']

In [14]:
collection.selected_layers = ['v171_geo_terms','v169_named_entities','v166_sentences','v166_morph_analysis']

In [28]:
from estnltk.vabamorf.morf import synthesize

#q = LayerQuery('v166_morph_analysis', lemma='jõgi')
cases = [
    ('n', 'nimetav'),
    ('g', 'omastav'),
    ('p', 'osastav'),
    ('ill', 'sisseütlev'),
    ('in', 'seesütlev'),
    ('el', 'seestütlev'),
    ('all', 'alaleütlev'),
    ('ad', 'alalütlev'),
    ('abl', 'alaltütlev'),
    ('tr', 'saav'),
    ('ter', 'rajav'),
    ('es', 'olev'),
    ('ab', 'ilmaütlev'),
    ('kom', 'kaasaütlev')]

all_forms = []
for case, name in cases:
    all_forms.append(', '.join(synthesize('kanal', 'sg ' + case, 'S')))
    all_forms.append(', '.join(synthesize('kanal', 'pl ' + case, 'S')))

In [30]:
from estnltk.storage.postgres import SubstringQuery, LayerQuery

#We are not sampling sentences but instead texts and then taking the first sentence that fits criteria
sampled_sentences = []
q = SubstringQuery('kanal')
for text_id, text_obj in collection.select(query=q, layers=['v171_geo_terms','v166_sentences']).sample(amount=50,amount_type='SIZE'):
    for term in text_obj.v171_geo_terms:
        if term.text in all_forms:
            for sentence in text_obj.v166_sentences:
                if sentence.end > term.start:
                    sampled_sentences.append(sentence)
                    break

In [ ]:
#Getting sentences to labelstudio format

In [37]:
from estnltk.converters.label_studio.label_studio import export_texts

In [38]:
from estnltk import Text

sampled_texts = []
for snt in sampled_sentences:
    sampled_texts.append(Text(' '.join(snt.text)))

In [46]:
from estnltk.taggers.system.rule_taggers import SubstringTagger,Ruleset,StaticExtractionRule

ruleset = Ruleset()

In [47]:
ruleset.add_rules([StaticExtractionRule('kanal')])

In [48]:
tagger = SubstringTagger(ruleset=ruleset)

In [49]:
for txt in sampled_texts:
    tagger.tag(txt)

In [53]:
layer_list = []
for txt in sampled_texts:
    layer_list.append(txt.terms)

In [55]:
export_texts('ls_converted.json',sampled_texts,layer_list)